<a href="https://colab.research.google.com/github/San-Di/scratchML/blob/master/ScratchNN-final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from enum import Enum
import tensorflow as tf

np.random.seed(1)

In [3]:
class ActivationType(Enum):
  Relu = lambda x: np.max(X, 0)
  Softmax = lambda x: np.exp(x)/sum(np.exp(x))
  Sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [4]:
def forward_propagation(X, W, b, g):
  # print("X",X.shape,"W", W.shape)
  z = np.dot(X, W) + b
  a = g(z)
  return z,a


In [5]:
def sigmoid_gradient(self, z):
  a = self._sigmoid(z)
  return a * (1 - a)


In [6]:
def backward_propagation(params, curr_layer, Y, m, last_layer):
  A_prev = params["A"+str(curr_layer - 1)]
  
  dZ = []
  if curr_layer == last_layer:
    # print("****** A{} = {}".format(curr_layer, params["A"+str(curr_layer)].shape) )
    dZ = params["A"+str(curr_layer)] - Y
  else:
    # print("W{} = {} \t dZ{} = {}".format(curr_layer, A_prev.shape, curr_layer, dZ.shape))
    dZ = np.dot(params['dZ' + str(curr_layer + 1)], params['W' + str(curr_layer + 1)].T) 
  # print("A{} = {} \t Z{} = {}".format(curr_layer, A_prev.shape, curr_layer, dZ.shape))
  dW = 1/m * np.dot( A_prev.T, dZ )
  db = 1/m * np.sum(dZ, axis=0, keepdims=True)
  return dZ, dW, db


In [7]:
def calculate_cost(params, Y, m):
  Y = np.array(Y)
  cost = -(1/m) * np.sum(Y * np.log(params["A4"]) + (1-Y) * np.log(1-params["A4"]))
  return cost


In [34]:
def fc(input, layer_dim: list, action: ActivationType, Y, m):
  parameters = {}
  X = input.reshape(input.shape[0],-1)
  dim = layer_dim
  dim.insert(0, X.shape[1])
  J_reg = 0
  parameters["A0"] = X
  for l in range(1, len(dim)):
    parameters["W"+str(l)] = np.random.randn(dim[l-1], dim[l]) * 0.01
    parameters["b"+str(l)] = np.zeros((1, dim[l]))

    Z, A = forward_propagation(parameters["A"+str(l-1)], parameters["W"+str(l)], parameters["b"+str(l)], ActivationType.Sigmoid)
    parameters["Z"+str(l)] = Z
    parameters["A"+str(l)] = A
    # print("{} = W -> {}\t b -> {}\t z -> {}\t A -> {}".format(l, parameters["W"+str(l)].shape, parameters["b"+str(l)].shape, parameters["Z"+str(l)].shape, parameters["A"+str(l)].shape))
    
  for l in range(len(dim) - 1, 0, -1):
    # print("reversed L ", l)
    dZ, dW, db = backward_propagation(parameters, l, Y, m, len(dim)-1)
    parameters["dZ"+str(l)] = dZ
    parameters["dW"+str(l)] = dW
    parameters["db"+str(l)] = db
    # print("dZ{} = {}\t dW{} = {}\t ".format(l, parameters["dZ"+str(l)].shape, l, parameters["dW"+str(l)].shape))
  
  for i in range(1, len(dim)):
    # print("{} = dW -> {}\t db -> {}\t dz -> {}\t A -> {}".format(i, parameters["dW"+str(i)].shape, parameters["db"+str(i)].shape, parameters["dZ"+str(i)].shape, parameters["A"+str(i)].shape))
    parameters["W"+str(i)] -= 0.01 * parameters["dW"+str(i)]
    parameters["b"+str(i)] -= 0.01 * parameters["db"+str(i)]

  J = calculate_cost(parameters, Y, m)

  for l in range(1, len(dim)):
    J_reg += (0.001 / (2 * 60000)) * (np.sum(parameters['W' + str(l)] ** 2))
    J += J_reg
  
  # accuracy
  ypred = np.round(parameters['A4'])
  # print(" predict => ", ypred.argmax(), " actual => ",y_i.argmax())
  return parameters

In [38]:
image = np.random.randn(10,64,64,3)

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255

Y = np.zeros((y_train.shape[0], 10))
for idnx in range(y_train.shape[0]):
  active_indx = y_train[idnx]
  Y[idnx][active_indx] = 1
print( y_train.shape, np.unique(y_train), y_train[1])
for epoch in range(100):
  acc = 0
  for i in range (60000):
    x_i = x_train[i, :]
    y_i = Y[i, :]
    # x_i = x_i.reshape((len(x_i), 1))
    # y_i = y_i.reshape((len(y_i), 1))
    param = fc(x_i, [40,30,20,10], ActivationType.Softmax, y_i, x_train.shape[0])
    ypred = np.round(param['A4'])
    if ypred.argmax() == y_i.argmax():
      acc += 1
  print("Epoch ", epoch, " acc ==> ", round(acc * 100 / x_train.shape[0], 3))

(60000,) [0 1 2 3 4 5 6 7 8 9] 0
Epoch  0  acc ==>  10.197
Epoch  1  acc ==>  10.302
Epoch  2  acc ==>  10.187
Epoch  3  acc ==>  10.317
Epoch  4  acc ==>  10.367
Epoch  5  acc ==>  10.317
Epoch  6  acc ==>  10.312
Epoch  7  acc ==>  10.232
Epoch  8  acc ==>  10.298
Epoch  9  acc ==>  10.362
Epoch  10  acc ==>  10.212
Epoch  11  acc ==>  10.088
Epoch  12  acc ==>  10.295
Epoch  13  acc ==>  9.988
Epoch  14  acc ==>  10.06
Epoch  15  acc ==>  10.41
Epoch  16  acc ==>  10.408
Epoch  17  acc ==>  10.068
Epoch  18  acc ==>  10.017
Epoch  19  acc ==>  10.218
Epoch  20  acc ==>  10.135
Epoch  21  acc ==>  10.217
Epoch  22  acc ==>  10.19
Epoch  23  acc ==>  10.07
Epoch  24  acc ==>  10.027
Epoch  25  acc ==>  10.132
Epoch  26  acc ==>  10.377
Epoch  27  acc ==>  10.277
Epoch  28  acc ==>  10.098
Epoch  29  acc ==>  10.4
Epoch  30  acc ==>  10.048
Epoch  31  acc ==>  10.1
Epoch  32  acc ==>  10.198
Epoch  33  acc ==>  10.443
Epoch  34  acc ==>  10.273
Epoch  35  acc ==>  10.288
Epoch  36  acc